In [2]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D, BatchNormalization,Input,concatenate, AveragePooling2D
import cv2
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from keras.models import Sequential
#from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
#from keras.preprocessing.image import ImageDataGenerator
#from keras.models import load_model
from sklearn.cluster import KMeans
import imutils
from skimage.feature import hog
import glob
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

c:\Users\PhucHai\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\PhucHai\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
root_path = 'D:\\CS231\\Final_Project'
root_list = os.listdir(root_path)
train_path = os.path.join(root_path, 'train')
test_path = os.path.join(root_path, 'test')

In [4]:
import os
import glob
import cv2
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from concurrent.futures import ThreadPoolExecutor
import numpy as np

image_train = []
y_train = []

def compute_hog_feature(image, orientations, pixels_per_cell, cells_per_block):
    if image is not None:
        hog_feature, _ = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
        return hog_feature
    return None

for index, image_dir in enumerate(os.listdir(train_path)):
    label = index
    image_dir_path = os.path.join(train_path, image_dir)
    image_paths = glob.glob(os.path.join(image_dir_path, "*.jpg"))
    
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            img_GRAY = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            img_GRAY = cv2.resize(img_GRAY, (128, 128))
            image_train.append(img_GRAY)
            y_train.append(label)

orientations_list = [2, 6, 9, 12]
pixels_per_cell_list = [(4, 4), (8, 8), (16, 16)]
cells_per_block_list = [(2, 2), (3, 3), (4, 4)]

best_params = None
best_accuracy = 0

def evaluate_hog_parameters(orientations, pixels_per_cell, cells_per_block):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(compute_hog_feature, img, orientations, pixels_per_cell, cells_per_block) for img in image_train]
        X_train_hog = [future.result() for future in futures if future.result() is not None]

    rf_classifier = RandomForestClassifier()

    param_grid = {'n_estimators': range(50, 151, 10)}

    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
   
    grid_search = GridSearchCV(rf_classifier, param_grid, cv=kf)
    
    grid_search.fit(X_train_hog, y_train)

    results = grid_search.cv_results_

    best_mean_score = -1
    best_parameters = None
    for mean_score, params in zip(results['mean_test_score'], results['params']):
        if best_mean_score < mean_score:
            best_mean_score = mean_score
            best_parameters = params

    return best_mean_score, best_parameters, orientations, pixels_per_cell, cells_per_block

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(evaluate_hog_parameters, orientations, pixels_per_cell, cells_per_block)
               for orientations in orientations_list
               for pixels_per_cell in pixels_per_cell_list
               for cells_per_block in cells_per_block_list]
    
    for future in futures:
        mean_score, params, orientations, pixels_per_cell, cells_per_block = future.result()
        print(f"Testing orientations={orientations}, pixels_per_cell={pixels_per_cell}, cells_per_block={cells_per_block}")
        print(f"Best accuracy: {mean_score}")
        if mean_score > best_accuracy:
            best_accuracy = mean_score
            best_params = {
                'orientations': orientations,
                'pixels_per_cell': pixels_per_cell,
                'cells_per_block': cells_per_block,
                'n_estimators': params['n_estimators']
            }

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")


Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(2, 2)
Best accuracy: 0.7231162322187553
Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(3, 3)
Best accuracy: 0.7701791022774734
Testing orientations=2, pixels_per_cell=(4, 4), cells_per_block=(4, 4)
Best accuracy: 0.7904835024445372
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(2, 2)
Best accuracy: 0.8157322556076949
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(3, 3)
Best accuracy: 0.8543817409036188
Testing orientations=2, pixels_per_cell=(8, 8), cells_per_block=(4, 4)
Best accuracy: 0.8573397538264993
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(2, 2)
Best accuracy: 0.8276183573692357
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(3, 3)
Best accuracy: 0.863786453086996
Testing orientations=2, pixels_per_cell=(16, 16), cells_per_block=(4, 4)
Best accuracy: 0.8583445937645873
Testing orientations=6, pixels_per_cell=(4, 4), ce